# 1. Setup Credentials

In [14]:
# @hidden_cell
# The project token is an authorization token that is used to access project resources like data sources, connections, and used by platform APIs.
from project_lib import Project
project = Project(project_id='', project_access_token='')
pc = project.project_context


In [1]:
tm1_credentials = {
    "address":"", 
    "port": "",
    "user": "", 
    "password": "", 
    "ssl": ""
}

# 2. Connect to Planning Analytics and Extract Data

In [2]:
!pip install TM1py

     |████████████████████████████████| 153kB 6.8MB/s eta 0:00:01
  Stored in directory: /home/dsxuser/.cache/pip/wheels/7c/fa/08/d56bd0c618b62fa0e0bc1d076cd7478deb25c73a042802eef5
Successfully built TM1py


In [3]:
from TM1py.Services import TM1Service
from TM1py.Utils.Utils import build_pandas_dataframe_from_cellset

In [4]:
# Import data from PA 
with TM1Service(address=tm1_credentials['address'], port=tm1_credentials['port'], user=tm1_credentials['user'], password=tm1_credentials['password'],ssl=tm1_credentials['ssl'] ) as tm1: 
    data = tm1.cubes.cells.execute_view(cube_name='OPS Sales', view_name='Training View', private = False)
    df = build_pandas_dataframe_from_cellset(data, multiindex=False)

In [5]:
df.head()

,Time Date,Version,Account,Product,Store,OPS Sales Source,OPS Sales Measure,Values
2190,01012018,Actual,4999,2477202,NSW_CW_ST0125,Base,Amount,1584.0
365,01012018,Actual,4999,2477202,QLD_CW_ST0005,Base,Amount,1936.0
4015,01012018,Actual,4999,2477202,VIC_CW_ST0411,Base,Amount,2080.0
1825,01012018,Actual,4999,2511675,NSW_CW_ST0125,Base,Amount,1861.2
0,01012018,Actual,4999,2511675,QLD_CW_ST0005,Base,Amount,2274.8


# 3. Preprocess Data

In [6]:
import pandas as pd

In [7]:
# Date Preprocessing - reformat date string
df['Time Date'] = df['Time Date'].apply(lambda x: x[4:]+x[2:4]+x[:2])

# Convert to Datetime
df['Time Date'] = pd.to_datetime(df['Time Date'])

In [8]:
df['Day'] = df['Time Date'].apply(lambda x: x.day)
df['Day of Week'] = df['Time Date'].apply(lambda x: x.dayofweek)
df['Day of Year'] = df['Time Date'].apply(lambda x: x.dayofyear)
df['Month'] = df['Time Date'].apply(lambda x: x.month)
df['Quarter End'] = df['Time Date'].apply(lambda x: x.is_quarter_end)
df['Month End'] = df['Time Date'].apply(lambda x: x.is_month_end)
df['Year'] = df['Time Date'].apply(lambda x: x.year)

In [9]:
df.head()

,Time Date,Version,Account,Product,Store,OPS Sales Source,OPS Sales Measure,Values,Day,Day of Week,Day of Year,Month,Quarter End,Month End,Year
2190,2018-01-01,Actual,4999,2477202,NSW_CW_ST0125,Base,Amount,1584.0,1,0,1,1,False,False,2018
365,2018-01-01,Actual,4999,2477202,QLD_CW_ST0005,Base,Amount,1936.0,1,0,1,1,False,False,2018
4015,2018-01-01,Actual,4999,2477202,VIC_CW_ST0411,Base,Amount,2080.0,1,0,1,1,False,False,2018
1825,2018-01-01,Actual,4999,2511675,NSW_CW_ST0125,Base,Amount,1861.2,1,0,1,1,False,False,2018
0,2018-01-01,Actual,4999,2511675,QLD_CW_ST0005,Base,Amount,2274.8,1,0,1,1,False,False,2018


In [10]:
# Update product column
df['Product'] = 'PROD'+df['Product']

In [11]:
df.drop(['Time Date', 'Version', 'Account', 'OPS Sales Source', 'OPS Sales Measure'], axis=1, inplace=True)

In [12]:
df.head()

,Product,Store,Values,Day,Day of Week,Day of Year,Month,Quarter End,Month End,Year
2190,PROD2477202,NSW_CW_ST0125,1584.0,1,0,1,1,False,False,2018
365,PROD2477202,QLD_CW_ST0005,1936.0,1,0,1,1,False,False,2018
4015,PROD2477202,VIC_CW_ST0411,2080.0,1,0,1,1,False,False,2018
1825,PROD2511675,NSW_CW_ST0125,1861.2,1,0,1,1,False,False,2018
0,PROD2511675,QLD_CW_ST0005,2274.8,1,0,1,1,False,False,2018


# 4. Export Data to CSV inside of Watson Studio Project

In [ ]:
project.save_data('trainingdata.csv', df.to_csv(index=False))